# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
!pip install dotenv

In [3]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [4]:
!pip install pandas

In [5]:
import pandas as pd
df = pd.read_csv('data/Data.csv')

In [6]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\r\n,I loved this product. But they only seem to l...


## LLMChain

In [7]:
df.info


<bound method DataFrame.info of                       Product  \
0        Queen Size Sheet Set   
1      Waterproof Phone Pouch   
2         Luxury Air Mattress   
3              Pillows Insert   
4   Milk Frother Handheld\r\n   
5     L'Or Espresso Café \r\n   
6  Hervidor de Agua Eléctrico   

                                              Review  
0  I ordered a king size set. My only criticism w...  
1  I loved the waterproof sac, although the openi...  
2  This mattress had a small hole in the top of i...  
3  This is the best throw pillow fillers on Amazo...  
4   I loved this product. But they only seem to l...  
5  Je trouve le goût médiocre. La mousse ne tient...  
6  Está lu bonita calienta muy rápido, es muy fun...  >

In [8]:
!pip install langchain_community

# SimpleSequentialChain
# Step 1: Basic LLMChain – Describe a Product in Detail

In [9]:

from langchain_openai import ChatOpenAI 
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

#  Set up the LLM model
llm = ChatOpenAI(temperature=0.2)  # lower temp = more focused/deterministic

# 📝 Create a simple prompt asking to describe a product
prompt = ChatPromptTemplate.from_template("Describe in detail this product: {product}")

#  Create the LLMChain
chain = LLMChain(llm=llm, prompt=prompt)

#  Run the chain for a single product
product = "Waterproof Phone Pouch"
output = chain.run(product)
print(output)


C:\Users\44758\AppData\Local\Temp\ipykernel_13280\1447550490.py:12: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)
C:\Users\44758\AppData\Local\Temp\ipykernel_13280\1447550490.py:16: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  output = chain.run(product)


A waterproof phone pouch is a protective case designed to keep your phone safe and dry while you are near water. It is typically made of a durable, waterproof material such as PVC or TPU that seals tightly to prevent water from seeping in. The pouch is transparent, allowing you to still use your phone's touchscreen and camera while it is inside.

The pouch usually features a secure closure mechanism, such as a zip-lock seal or a snap closure, to ensure that water cannot penetrate the case. Some pouches also come with additional features such as a lanyard or wrist strap for easy carrying, and a built-in headphone jack for listening to music while your phone is inside.

Waterproof phone pouches are ideal for activities such as swimming, snorkeling, kayaking, or simply lounging by the pool or beach. They provide peace of mind knowing that your phone is protected from water damage, allowing you to capture photos and videos without worrying about potential water exposure.

Overall, a waterp

## 🔁Step 2: SimpleSequentialChain – Name ➡ Description Chain


In [10]:
from langchain.chains import SimpleSequentialChain

#  Use a more creative LLM for name generation
llm = ChatOpenAI(temperature=0.9)

# ✨ First Prompt: Generate catchy product name
first_prompt = ChatPromptTemplate.from_template(
    "Generate a catchy product name for the following item: {product}"
)
chain_one = LLMChain(llm=llm, prompt=first_prompt)

# ✨ Second Prompt: Create a short product description
second_prompt = ChatPromptTemplate.from_template(
    "Write a short and engaging product description for: {product}"
)
chain_two = LLMChain(llm=llm, prompt=second_prompt)

# Connect both into a SimpleSequentialChain
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two], verbose=True)

# Run the chain for a sample product
overall_simple_chain.run("Milk Frother Handheld")




> Entering new SimpleSequentialChain chain...
WhipN'Whirl Handheld Milk Frother
Introducing the WhipN'Whirl Handheld Milk Frother, the perfect tool for creating creamy, frothy drinks in the comfort of your own home. This easy-to-use frother is compact, portable, and powerful, making it ideal for whipping up lattes, cappuccinos, and hot chocolates. Simply insert the frother into your milk and watch as it transforms into a velvety foam in seconds. Say goodbye to expensive coffee shop drinks and hello to barista-quality beverages with the WhipN'Whirl Handheld Milk Frother. Upgrade your morning routine today!

> Finished chain.


"Introducing the WhipN'Whirl Handheld Milk Frother, the perfect tool for creating creamy, frothy drinks in the comfort of your own home. This easy-to-use frother is compact, portable, and powerful, making it ideal for whipping up lattes, cappuccinos, and hot chocolates. Simply insert the frother into your milk and watch as it transforms into a velvety foam in seconds. Say goodbye to expensive coffee shop drinks and hello to barista-quality beverages with the WhipN'Whirl Handheld Milk Frother. Upgrade your morning routine today!"

**Repeat the above twice for different products**

In [11]:
# from data
product_list = [
    "Queen Size Sheet Set",
    "Luxury Air Mattress"
]

for product in product_list:
    print(f"\n Generating content for: {product}")
    overall_simple_chain.run(product)


 Generating content for: Queen Size Sheet Set


> Entering new SimpleSequentialChain chain...
Royal Dream Comfort Queen Sheet Set
Indulge in luxurious comfort with the Royal Dream Comfort Queen Sheet Set. Made from ultra-soft microfiber material, these sheets are designed to provide a peaceful night's sleep. The set includes a fitted sheet, flat sheet, and two pillowcases, all featuring a stylish and elegant design fit for royalty. Treat yourself to the ultimate relaxation experience with the Royal Dream Comfort Queen Sheet Set.

> Finished chain.

 Generating content for: Luxury Air Mattress


> Entering new SimpleSequentialChain chain...
CloudComfort AirLuxury
Introducing CloudComfort AirLuxury - the ultimate in luxurious comfort for your home. Experience the unparalleled softness and support of our premium air mattress, designed to give you a restful night’s sleep every time. Made with high-quality materials and advanced technology, CloudComfort AirLuxury is the perfect blend of lu

## SequentialChain - Translation

In [12]:
df

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\r\n,I loved this product. But they only seem to l...
5,L'Or Espresso Café \r\n,Je trouve le goût médiocre. La mousse ne tient...
6,Hervidor de Agua Eléctrico,"Está lu bonita calienta muy rápido, es muy fun..."


In [13]:
df["Review"][1]

'I loved the waterproof sac, although the opening was made of a hard plastic. I don’t know if that would break easily. But I couldn’t turn my phone on, once it was in the pouch.'

In [14]:
from langchain.chains import SequentialChain

In [15]:
# 🌍 Prompt 1: Translate the review into Italian
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review into Italian: {review}"
)

# 🔗 Chain 1: Takes input 'review', outputs 'italian_review'
chain_one = LLMChain(
    llm=llm,
    prompt=first_prompt,
    output_key='italian_review'
)



In [16]:
#  Prompt 2: Summarize the Italian review
second_prompt = ChatPromptTemplate.from_template(
    "Summarize the following review in the same language of the review itself: {italian_review}"
)

#  Chain 2: Takes 'italian_review', outputs 'italian_summary'
chain_two = LLMChain(
    llm=llm,
    prompt=second_prompt,
    output_key='italian_summary'
)



In [17]:
# Prompt 3: Translate the summary to English 
third_prompt = ChatPromptTemplate.from_template(
    "Translate the following to English if it is not in English already, otherwise say 'we are done': {italian_summary}"
)

# Chain 3: Takes 'italian_summary', outputs 'english_translation'
chain_three = LLMChain(
    llm=llm,
    prompt=third_prompt,
    output_key='english_translation'
)



In [18]:
#  Prompt 4: Follow-up message based on summary + English version
fourth_prompt = ChatPromptTemplate.from_template(
    "Grazie per la tua recensione! Riassunto: {italian_summary}. Traduzione in inglese: {english_translation}."
)

# Chain 4: Final message
chain_four = LLMChain(
    llm=llm,
    prompt=fourth_prompt,
    output_key='followup_message'
)


# 🧠 Full chain that runs: Translate → Summarize → Re-translate

In [19]:
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three],
    input_variables=["review"],
    output_variables=[
        "italian_review",
        "italian_summary",
        "english_translation"
    ],
    verbose=True
)


In [20]:
review = df.Review[1]
print(review)

I loved the waterproof sac, although the opening was made of a hard plastic. I don’t know if that would break easily. But I couldn’t turn my phone on, once it was in the pouch.


In [21]:
overall_chain(review)



> Entering new SequentialChain chain...


C:\Users\44758\AppData\Local\Temp\ipykernel_13280\2289504204.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  overall_chain(review)



> Finished chain.


{'review': 'I loved the waterproof sac, although the opening was made of a hard plastic. I don’t know if that would break easily. But I couldn’t turn my phone on, once it was in the pouch.',
 'italian_review': "Ho amato il sacchetto impermeabile, anche se l'apertura era fatta di plastica dura. Non so se si romperebbe facilmente. Ma non riuscivo ad accendere il mio telefono, una volta che era nella tasca.",
 'italian_summary': "Mi è piaciuto il sacchetto impermeabile, anche se l'apertura era in plastica rigida. Non so se si romperebbe facilmente. Non riuscivo ad usare il mio telefono una volta dentro la tasca.",
 'english_translation': "I liked the waterproof bag, even though the opening was in hard plastic. I'm not sure if it would break easily. I couldn't use my phone once inside the pocket."}

## Router Chain
- A Router Chain is like an AI switchboard  — it receives a question and decides which expert (chain) should handle it.

- Below are Prompt Templates (Not Chains Yet)
Each of templates describes a persona (expert) the AI will "act like" for a given topic.

In [22]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

biology_template = """You are an excellent biologist. \
You have a deep understanding of living organisms, \
from the molecular and cellular level to entire ecosystems. \
You are skilled at observing patterns in nature, analyzing biological data, \
and explaining complex processes like evolution, genetics, physiology, and ecology. \
You can clearly communicate how life functions and adapts, \
and you make connections between different biological concepts \
to answer challenging questions.

Here is a question:
{input}"""

In [23]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    },
    {
        "name": "biology",
        "description": "Good for answering biology questions",
        "prompt_template": biology_template
    }
]

In [24]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [25]:
llm = ChatOpenAI(temperature=0)

In [26]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [27]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [28]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [29]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [30]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

C:\Users\44758\AppData\Local\Temp\ipykernel_13280\3038952769.py:1: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  chain = MultiPromptChain(router_chain=router_chain,


In [31]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an idealized physical body that absorbs all incident electromagnetic radiation and emits radiation at all frequencies. The radiation emitted by a black body depends only on its temperature and follows a specific distribution known as Planck's law. This type of radiation is important in understanding concepts such as thermal radiation and the behavior of objects at different temperatures."

In [32]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'The answer to 2 + 2 is 4.'

In [33]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
biology: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


"Every cell in our body contains DNA because DNA is the genetic material that carries the instructions for the development, functioning, and reproduction of all living organisms. DNA contains the information needed to build and maintain an organism, including the proteins that make up our cells and tissues. \n\nHaving DNA in every cell ensures that each cell has the necessary genetic information to carry out its specific functions and to replicate itself accurately during cell division. This ensures that the genetic information is passed on to the next generation of cells, maintaining the integrity and continuity of the organism's genetic code.\n\nAdditionally, DNA serves as a storage system for genetic information, allowing for the transmission of traits from one generation to the next through the process of inheritance. This is essential for the survival and evolution of species, as it allows for genetic variation and adaptation to changing environments over time. \n\nIn summary, eve

**Repeat the above at least once for different inputs and chains executions - Be creative!**

In [35]:
chain.run("What is Newton's second law?")



> Entering new MultiPromptChain chain...
physics: {'input': "What is Newton's second law?"}
> Finished chain.


"Newton's second law states that the acceleration of an object is directly proportional to the net force acting on it and inversely proportional to its mass. In other words, the greater the force applied to an object, the greater its acceleration will be, and the more massive an object is, the less it will accelerate for a given force. Mathematically, this can be expressed as F = ma, where F is the net force, m is the mass of the object, and a is the acceleration."

In [36]:
chain.run("What is the difference between a virus and a bacteria?")



> Entering new MultiPromptChain chain...
biology: {'input': 'What is the difference between a virus and a bacteria?'}
> Finished chain.


"Viruses and bacteria are both types of microorganisms, but they have some key differences in terms of their structure, behavior, and how they interact with living organisms.\n\n1. Structure: Bacteria are single-celled organisms that have a cell wall, cell membrane, and genetic material (DNA or RNA) contained within their cytoplasm. They can reproduce independently through binary fission. Viruses, on the other hand, are not considered living organisms because they lack cellular structure. They consist of genetic material (DNA or RNA) enclosed in a protein coat called a capsid.\n\n2. Reproduction: Bacteria reproduce by binary fission, where a single bacterial cell divides into two identical daughter cells. Viruses cannot reproduce on their own and must infect a host cell to replicate. They hijack the host cell's machinery to make copies of themselves, which can then infect other cells.\n\n3. Size: Bacteria are larger than viruses, typically ranging from 0.5 to 5 micrometers in size, whi

In [37]:
chain.run("how hard is computer science?")



> Entering new MultiPromptChain chain...
computer science: {'input': 'how hard is computer science?'}
> Finished chain.


"Computer science can be challenging, but it ultimately depends on the individual's dedication, passion, and problem-solving skills. With the right mindset and determination, anyone can succeed in the field of computer science. It requires a strong understanding of theories and algorithms, as well as excellent communication skills and the ability to collaborate with others. Overall, computer science can be as hard or as easy as you make it, but with the right skills and mindset, success is definitely achievable."

In [38]:
chain.run("how hard is aerospace engineering?")



> Entering new MultiPromptChain chain...
None: {'input': 'how hard is aerospace engineering?'}
> Finished chain.


'Aerospace engineering can be quite challenging, as it involves the design, development, and testing of aircraft, spacecraft, and other aerospace systems. It requires a strong foundation in mathematics, physics, and engineering principles, as well as a deep understanding of aerodynamics, propulsion, materials science, and other specialized areas. Additionally, aerospace engineering projects often involve complex systems and require collaboration with multidisciplinary teams. Overall, aerospace engineering can be a demanding field, but it can also be highly rewarding for those who are passionate about aviation and space exploration.'